In [4]:
!pip install river
!pip install microprediction

from river import linear_model
from river import preprocessing

In [9]:
from microprediction import new_key
new_key(difficulty=9)

'b37559781261a96aeab54a49ca3cfdbb'

In [10]:
from microprediction import MicroCrawler
from microprediction import MicroReader
import math 
import numpy as np
import river.stats
reader = MicroReader()

In [11]:
class MyCrawler(MicroCrawler):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.check = False
        self.season = 1
        self.var = river.stats.Var()
        
        
        
    def season(self, frequency):
        if frequency >= 86400:
#         The frequency is bigger than a day, then seasonality should be weekly
            return math.ceil(604800/frequency)
        elif frequency >= 3600:
#         The frequency is bigger than an hour but less tha a day, then seasonality should be hourly
            return math.ceil(86400/frequency)
        elif frequency >= 60:
#         The frequency is bigger than a minute but less than an hour, then seasonality should be minute
            return math.ceil(3600/frequency)
        else:
#         The frequency is less than a minute, then seasonality is second
            return math.ceil(60/frequency)

    def number_of_element_in_season(self, lagged_time_array):
        if len(lagged_time_array) == 1:
            return season(lagged_time_array[0])
        else:
            return season(lagged_time_array[0] - lagged_time_array[1])
    
        
    def sample(self, lagged_values, lagged_times=None, name=None, delay=None, **ignored):
        """ An example of a sample method. This is where all the intelligence goes
               :param lagged_values [ float ]    List with most recent listed first
               :param lagged_times [ float ]    List of epoch times, most recent listed first
               :param name          str         Name of stream
               :param delay         int         Prediction horizon in seconds
               **ignored                        For future compatability.
               :returns            [ float ]    A vector of numbers somewhat indicative of a probability density, of length self.num_predictions
        """
        lagged_times = reader.get_lagged_times(name = name)
        if not self.check:
            self.season = self.number_of_element_in_season(lagged_times)
            self.check = True
            
        queue = collections.deque([], self.season)
            
            
        
        model = (preprocessing.StandardScaler() | linear_model.LinearRegression(intercept_lr=.1))
        
        
        
        
        
        
        for x, y in zip(lagged_times, lagged_values):
            
        
            new_feats = {f"lag_{i}": v for i, v in enumerate(queue)}

            # copy of x
            x_ = dict(x)
            x_.update(new_feats)

            y_pred = model.predict_one(x_)
            model.learn_one(x_, y)
            queue.append(y)
            variance = self.var.update(y_pred).get()




        return np.random.normal(y_pred, variance, 255)